In [ ]:
%install '.package(path: "$cwd/FastaiNotebooks")' FastaiNotebooks

Installing packages:
	.package(path: "/home/ubuntu/notebooks/swift/FastaiNotebooks")
		FastaiNotebooks
Working in: /tmp/tmpo9ipn8va
Fetching https://github.com/mxcl/Path.swift
Fetching https://github.com/JustHTTP/Just
Completed resolution in 4.50s
Cloning https://github.com/mxcl/Path.swift
Resolving https://github.com/mxcl/Path.swift at 0.16.2
Cloning https://github.com/JustHTTP/Just
Resolving https://github.com/JustHTTP/Just at 0.7.1
Compile Swift Module 'Just' (1 sources)
Compile Swift Module 'Path' (9 sources)
Compile Swift Module 'FastaiNotebooks' (3 sources)
Compile Swift Module 'jupyterInstalledPackages' (1 sources)
Linking ./.build/x86_64-unknown-linux/debug/libjupyterInstalledPackages.so
Installation complete!

In [ ]:
import FastaiNotebooks

In [ ]:
// export
import Path
import TensorFlow

## The forward and backward passes

In [ ]:
// export
public func normalize(_ x:Tensor<Float>, mean:Tensor<Float>, std:Tensor<Float>) -> Tensor<Float> {
    return (x-mean)/std
}

In [ ]:
let mnist = MnistDataset(path: Path.home/".fastai"/"data"/"mnist_tst")
var x_train = mnist.xTrain
var y_train = mnist.yTrain
var x_valid = mnist.xValid
var y_valid = mnist.yValid

There is no std method to Tensor in swift yet, so we add it as an extension:

In [ ]:
//export
public extension Tensor where Scalar:FloatingPoint {
    func std() -> Tensor<Scalar> {
        return sqrt(self.variance(alongAxes: Array(self.shape.indices)).flattened()[0])
    }
}

Normalize the training and validation sets.

In [ ]:
let train_mean = x_train.mean()
let train_std  = x_train.std()

In [ ]:
x_train = normalize(x_train, mean: train_mean, std: train_std)
x_valid = normalize(x_valid, mean: train_mean, std: train_std)

In [ ]:
//export
public func test_near_zero(_ a:Tensor<Float>, tol:Float=1e-3) {
    assert(Raw.abs(a)<tol, "Near zero: \(a)")
}

In [ ]:
test_near_zero(x_train.mean())
test_near_zero(x_train.std() - 1.0)

In [ ]:
x_train.shape

▿ TensorShape
  ▿ dimensions : 2 elements
    - 0 : 60000
    - 1 : 784


In [ ]:
let (n,m) = (x_train.shape[0],x_train.shape[1])
let c = y_train.max()+1
print(n,m,c)

60000 784 10


## Foundations version

### Basic architecture

In [ ]:
//num hidden
let nh:Int32 = 50

In [ ]:
// simplified kaiming init / he init
let w1:Tensor<Float> = Tensor(randomNormal: [m,nh]) / sqrt(Float(m))
let b1:Tensor<Float> = Tensor(repeating: 0.0, shape: [nh])
let w2:Tensor<Float> = Tensor(randomNormal: [nh,1]) / sqrt(Float(nh))
let b2:Tensor<Float> = Tensor(repeating: 0.0, shape: [1])

In [ ]:
test_near_zero(w1.mean())
test_near_zero(w1.std()-1/sqrt(Float(m)))

In [ ]:
// This should be ~ (0,1) (mean,std)...
(x_valid.mean(),x_valid.std())

▿ 2 elements
  - .0 : 0.006017743
  - .1 : 1.0076997


In [ ]:
func lin(_ x:Tensor<Float>, _ w:Tensor<Float>, _ b:Tensor<Float>) ->Tensor<Float> {return matmul(x, w) + b}

In [ ]:
let t = lin(x_valid, w1, b1)

In [ ]:
//...so should this, because we used kaiming init, which is designed to do this
(t.mean(),t.std())

▿ 2 elements
  - .0 : 0.09107302
  - .1 : 1.0393536


In [ ]:
func relu(_ x:Tensor<Float>) -> Tensor<Float> {return max(x, 0)}

In [ ]:
let t = relu(lin(x_valid, w1, b1))

In [ ]:
//...actually it really should be this!
(t.mean(),t.std())

▿ 2 elements
  - .0 : 0.4556805
  - .1 : 0.6324954


In [ ]:
// kaiming init / he init for relu
let w1:Tensor<Float> = Tensor(randomNormal: [m,nh]) * sqrt(2.0/Float(m))

In [ ]:
(w1.mean(),w1.std())

▿ 2 elements
  - .0 : 5.81798e-05
  - .1 : 0.050485164


In [ ]:
let t = relu(lin(x_valid, w1, b1))
(t.mean(),t.std())

▿ 2 elements
  - .0 : 0.5542916
  - .1 : 0.82010573


In [ ]:
func model(_ xb: Tensor<Float>) -> Tensor<Float>{
    let l1 = lin(xb, w1, b1)
    let l2 = relu(l1)
    let l3 = lin(l2, w2, b2)
    return l3
}

In [ ]:
time(repeating: 10) {let _ = model(x_valid)}

0.669766 ms


### Loss function

In [ ]:
let preds = model(x_train)

In [ ]:
// export
func mse(_ out:Tensor<Float>, _ targ:Tensor<Float>) -> Tensor<Float> {
    return (out.squeezingShape(at: -1) - targ).squared().mean()
}

In [ ]:
func to_float<Scalar: TensorFlowScalar & StringProtocol>(_ x:Tensor<Scalar>) -> Tensor<Float>{
    return Tensor(shape: x.shape, scalars: x.array.scalars.compactMap{ Float($0)})
}

In [ ]:
var y_trainf = to_float(y_train)
var y_validf = to_float(y_valid)

In [ ]:
mse(preds, y_trainf)

28.24072


### Gradients and backward pass

In [ ]:
class TensorWithGrad {
    var inner: Tensor<Float>
    var grad:  Tensor<Float>
    
    init(_ x: Tensor<Float>) {
        inner = x
        grad = Tensor(repeating: 0.0, shape:x.shape)
    } 
}

In [ ]:
func lin(_ x:TensorWithGrad, _ w:TensorWithGrad, _ b:TensorWithGrad) -> TensorWithGrad {
    return TensorWithGrad(matmul(x.inner, w.inner) + b.inner)
}
func relu(_ x:TensorWithGrad) -> TensorWithGrad {return TensorWithGrad(max(x.inner, 0))}
func mse(_ inp: TensorWithGrad, _ targ : Tensor<Float>) -> Tensor<Float>{
    //grad of loss with respect to output of previous layer
    return (inp.inner.squeezingShape(at: -1) - targ).squared().mean()
}

In [ ]:
func mse_grad(_ inp: TensorWithGrad, _ targ : Tensor<Float>){
    //grad of loss with respect to output of previous layer
    inp.grad = (inp.inner.squeezingShape(at: -1) - targ).expandingShape(at: -1) / Float(inp.inner.shape[0])
}

In [ ]:
func relu_grad(_ inp:TensorWithGrad, _ out:TensorWithGrad){
    //grad of relu with respect to input activations
    inp.grad = (inp.inner .> 0).selecting(out.grad, Tensor<Float>(repeating:0.0, shape:inp.inner.shape))
}

In [ ]:
func lin_grad(_ inp:TensorWithGrad, _ out:TensorWithGrad, _ w:TensorWithGrad, _ b:TensorWithGrad){
    //grad of relu with respect to input activations
    inp.grad = matmul(out.grad, w.inner.transposed())
    w.grad = matmul(inp.inner.transposed(), out.grad)
    b.grad = out.grad.sum(squeezingAxes: 0)
}

In [ ]:
let w1a = TensorWithGrad(w1)
let b1a = TensorWithGrad(b1)
let w2a = TensorWithGrad(w2)
let b2a = TensorWithGrad(b2)

In [ ]:
func forward_and_backward(_ inp:TensorWithGrad, _ targ:Tensor<Float>){
    //forward pass:
    let l1 = lin(inp, w1a, b1a)
    let l2 = relu(l1)
    let out = lin(l2, w2a, b2a)
    //we don't actually need the loss in backward!
    let loss = mse(out, targ)
    
    //backward pass:
    mse_grad(out, targ)
    lin_grad(l2, out, w2a, b2a)
    relu_grad(l1, l2)
    lin_grad(inp, l1, w1a, b1a)
}

In [ ]:
let inp = TensorWithGrad(x_train)

In [ ]:
forward_and_backward(inp, y_trainf)

### Export

In [ ]:
export_notebooks(Path.cwd)

Converting 02_fully_connected.ipynb
Converting 00_load_data.ipynb
Converting 01_matmul.ipynb
